In [1]:
import pandas as pd
import re
from openpyxl import load_workbook

nomearq = r'C:\Users\Alexandre\Documents\github\Automacoes\leitura de txt RAZAO\dadosfictios'
mesarq= 'Junho'

In [2]:
with open(f"{nomearq}.txt", "r") as arquivo:
    dados = arquivo.read()

padrao_cabecalho = re.compile(r'LUIZA ASSESSORIA CONTABIL LTDA                    .*?Saldo', re.DOTALL)

# Substituir o cabeçalho por uma string vazia
texto_sem_cabecalho = re.sub(padrao_cabecalho, '', dados)
padrao_cabecalho = re.compile(r'-{131}', re.DOTALL)
texto_sem_cabecalho = re.sub(padrao_cabecalho, '', texto_sem_cabecalho)
padrao_cabecalho = re.compile(r'-{88}', re.DOTALL)
texto_sem_cabecalho = re.sub(padrao_cabecalho, '', texto_sem_cabecalho)
padrao_cabecalho = re.compile(r'-{50}', re.DOTALL)
texto_sem_cabecalho = re.sub(padrao_cabecalho, '', texto_sem_cabecalho)
texto_sem_cabecalho = re.sub(r'[\x00-\x08\x0B\x0C\x0E-\x1F\x7F-\x9F]', '', texto_sem_cabecalho)
texto_sem_cabecalho = texto_sem_cabecalho.split('\n')

df = pd.DataFrame(columns=['Data', 'lanca', 'contra','nome','debito', 'credito', 'saldo'])
anterior = 1

for e, linha in enumerate(texto_sem_cabecalho):
    if 'Saldo do Mes:' in linha:
        texto_sem_cabecalho.pop(e)

for e, linha in enumerate(texto_sem_cabecalho):
    if 'Saldo Atual:' in linha:
        texto_sem_cabecalho.pop(e)

for e, linha in enumerate(texto_sem_cabecalho):
    if 'Saldo Anterior:' in linha:
        if anterior == 1:
            anterior = linha
        texto_sem_cabecalho.pop(e)

for e, linha in enumerate(texto_sem_cabecalho):
    if 'Saldo Geral:' in linha:
        texto_sem_cabecalho.pop(e)

for e, linha in enumerate(texto_sem_cabecalho):
    if 'Sem Movimento' in linha:
        texto_sem_cabecalho.pop(e)

for e, linha in enumerate(texto_sem_cabecalho):
    if 'Conta Custo' in linha:
        texto_sem_cabecalho.pop(e)

for e, linha in enumerate(texto_sem_cabecalho):
    if '' == linha.strip:
        texto_sem_cabecalho.pop(e)

for e, linha in enumerate(texto_sem_cabecalho):
    if '' == linha:
        texto_sem_cabecalho.pop(e)

texto_sem_cabecalho = [linha for linha in texto_sem_cabecalho if not linha.isspace()]

linhas_organizadas = re.sub(r'\n {44}', '', '\n'.join(texto_sem_cabecalho), flags=re.DOTALL)
linhas_organizadas = linhas_organizadas.split('\n')

In [3]:
for e, linha in enumerate(linhas_organizadas):
    if len(linha.split()) < 4:
        linhas_organizadas.pop(e)

for linha in linhas_organizadas:
    linha_32 = linha[:33].strip().split()
    print(linha_32)
    data = linha_32[0]
    lanca = linha_32[1]
    contra = linha_32[2]
    linha_49 = linha[-53:].replace('C', ' ').replace('D', ' ')
    if any(char.isalpha() for char in linha_49):
        debito =  ''
        credito = ''
        saldo = ''
        nome = linha[33:].strip()
    else:
        debito =  linha_49[:16].strip().replace('.', '').replace(',', '')
        credito = linha_49[16:33].strip().replace('.', '').replace(',', '')
        saldo = linha_49[33:49].strip().replace('.', '').replace(',', '')
        nome = linha[33:-49].strip()
    
    df = pd.concat([df,pd.DataFrame([{'Data': data, 'lanca': lanca, 'contra': contra,'nome': nome,'debito': debito, 'credito': credito, 'saldo': saldo}])], ignore_index=True)

['65/67/8685', '6666698868', '6666666668']
['65/67/8685', '6666698885', '6666666668']
['65/67/8685', '6666698886', '6666666668']
['65/67/8685', '6666698887', '6666666668']
['65/67/8685', '6666698888', '6666666668']
['65/67/8685', '6666698889', '6666666668']
['65/67/8685', '6666698856', '6666666668']
['65/67/8685', '6666698855', '6666666668']
['65/67/8685', '6666698857', '6666666668']
['65/67/8685', '6666698855', '6666666668']
['65/67/8685', '6666698856', '6666666668']
['65/67/8685', '6666698857', '6666666668']
['65/67/8685', '6666698858', '6666666668']
['65/67/8685', '6666698859', '6666666668']
['65/67/8685', '6666698876', '6666666668']
['65/67/8685', '6666698879', '6666666668']
['65/67/8685', '6666698878', '6666666668']
['65/67/8685', '6666698875', '6666666668']
['65/67/8685', '6666698877', '6666666668']
['65/67/8685', '6666698875', '6666666668']
['65/67/8685', '6666698876', '6666666668']
['65/67/8685', '6666698877', '6666666668']
['65/67/8685', '6666698878', '6666666668']
['65/67/868

In [4]:
df = df.fillna(0)

def filtarNF(x):
    padrao = r"nf\s?(n.o)?(.)?(:)?(-)?\s?(\d+)"
    try:
        return re.search(padrao, x.lower())[0].replace('nf', '').replace('n.o', '').replace('-', '').replace(':', '').replace('.', '').strip()
    except:
        try:
            padrao = r"ft\s?(n.o)?(:)?(-)?\s?(\d+)"
            return re.search(padrao, x.lower())[0].replace('ft', '').replace('n.o', '').replace('-', '').replace(':', '').replace('.', '').strip()
        except:
            print(x)
            return '0000000'

df['NF'] = df['nome'].apply(lambda x: float(filtarNF(x)))

df = df[['Data', 'lanca', 'contra', 'NF', 'nome', 'debito', 'credito', 'saldo']]


In [7]:
df = df.fillna(0)
df['debito'] = pd.to_numeric(df['debito'])  
df['credito'] = pd.to_numeric(df['credito'])  
df['saldo'] = pd.to_numeric(df['saldo']) 
df = df.fillna(0)

df['contra'] = df['contra']


df['debito'] = df['debito'].apply(lambda x: float('{:.2f}'.format(x / 100)))
df['credito'] = df['credito'].apply(lambda x: float('{:.2f}'.format(x / 100)))
df['saldo'] = df['saldo'].apply(lambda x: float('{:.2f}'.format(x / 100)))

In [8]:
soma_df_deb = df['debito'].sum()
soma_df_cred = df['credito'].sum()
print(soma_df_deb, soma_df_cred)

71072441.27000001 68752651.82


In [9]:
df_tarifa = df[df['nome'].str.contains('TARIFA')]
df = df.drop(df_tarifa.index)
df_recebimento = df[df['nome'].str.lower().str.contains('recebimento|reecebimento')]
df = df.drop(df_recebimento.index)
df_pis = df[df['nome'].str.contains('PIS')]
df = df.drop(df_pis.index)
df_cofins = df[df['nome'].str.contains('COFINS')]
df = df.drop(df_cofins.index)
df_iss = df[df['nome'].str.contains('ISS')]
df = df.drop(df_iss.index)
df_IRRF = df[df['nome'].str.contains('IRRF')]
df = df.drop(df_IRRF.index)
df_CSLL = df[df['nome'].str.contains('CSLL')]
df = df.drop(df_CSLL.index)
df_inss = df[df['nome'].str.contains('INSS')]
df = df.drop(df_inss.index)
df_venda = df[df['nome'].str.contains('VENDA')]
df = df.drop(df_venda.index)

In [10]:
soma_recebimento = df_recebimento[['credito','debito']].sum()
soma_pis = df_pis[['credito','debito']].sum()
soma_cofins = df_cofins[['credito','debito']].sum()
soma_iss = df_iss[['credito','debito']].sum()
soma_IRRF = df_IRRF[['credito','debito']].sum()
soma_CSLL = df_CSLL[['credito','debito']].sum()
soma_inss = df_inss[['credito','debito']].sum()
soma_venda = df_venda[['credito','debito']].sum()
soma_tarifa = df_tarifa[['credito','debito']].sum()
soma_outros = df[['credito','debito']].sum()

In [11]:
totaldebito = sum([soma_pis['debito'], soma_cofins['debito'], soma_iss['debito'], soma_IRRF['debito'], soma_CSLL['debito'], soma_inss['debito']])
totalcredito = sum([soma_pis['credito'], soma_cofins['credito'], soma_iss['credito'], soma_IRRF['credito'], soma_CSLL['credito'], soma_inss['credito']])

anterior =  float(anterior.replace('Saldo Anterior:', '').replace('D', '').strip().replace('.', '').replace(',', '.'))

data = {
        'Tipos': ['RAZAO 1 CONTA 13', f'{mesarq}/2023', 'SALDO ANTERIOR', '', 'VENDAS NO MÊS', '', 'RETENÇÕES', 'PIS', 'COFINS', 'ISS', 'IRRF', 'CSLL', 'INSS', 'TOTAL', '', '', 'RECEBIMENTOS', 'TARIFA', '', '', 'TOTAL GERAL DO MÊS', '', '', '', 'OUTROS'],
        'debito': ['', '', anterior, 'ENTRADA', soma_venda['debito'], '', '', soma_pis['debito'], soma_cofins['debito'], soma_iss['debito'], soma_IRRF['debito'], soma_CSLL['debito'], soma_inss['debito'], totaldebito, '', '', soma_recebimento['debito'], soma_tarifa['debito'], '', '', totaldebito+soma_recebimento['debito']+soma_venda['debito']+soma_tarifa['debito'], '', '', '', soma_outros['debito']],
        'credito': ['', '', '', 'SAIDA', soma_venda['credito'], '', '', soma_pis['credito'], soma_cofins['credito'], soma_iss['credito'], soma_IRRF['credito'], soma_CSLL['credito'], soma_inss['credito'], totalcredito, '', '', soma_recebimento['credito'], soma_tarifa['credito'], '', '', totalcredito+soma_recebimento['credito']+soma_venda['credito']+soma_tarifa['credito'], '', '', '', soma_outros['credito']]}

dfarrumado = pd.DataFrame(data)
dfarrumado.replace(0.0, '', inplace=True)

In [12]:
display(dfarrumado)

,Tipos,debito,credito
0,RAZAO 1 CONTA 13,,
1,Junho/2023,,
2,SALDO ANTERIOR,86896795.88,
3,,ENTRADA,SAIDA
4,VENDAS NO MÊS,71072441.27,
5,,,
6,RETENÇÕES,,
7,PIS,,160240.98
8,COFINS,,741215.26
9,ISS,,745435.3


In [13]:
with pd.ExcelWriter(f'{nomearq}-resumo.xlsx') as writer:
    # Salve cada DataFrame em uma planilha separada
    dfarrumado.to_excel(writer, sheet_name='RESUMO', header=False ,index=False, float_format="%.2f")
    df_pis.to_excel(writer, sheet_name='pis', index=False, float_format="%.2f")
    df_cofins.to_excel(writer, sheet_name='cofins', index=False, float_format="%.2f")
    df_iss.to_excel(writer, sheet_name='iss', index=False, float_format="%.2f")
    df_IRRF.to_excel(writer, sheet_name='irrf', index=False, float_format="%.2f")
    df_CSLL.to_excel(writer, sheet_name='csll', index=False, float_format="%.2f")
    df_inss.to_excel(writer, sheet_name='insss', index=False, float_format="%.2f")
    df_venda.to_excel(writer, sheet_name='venda', index=False, float_format="%.2f")
    df_recebimento.to_excel(writer, sheet_name='recebimento', index=False, float_format="%.2f")
    df.to_excel(writer, sheet_name='outros', index=False, float_format="%.2f")
    # Adicione mais DataFrames conforme necessário

dffiltrado = df[df['contra'].isin([2,4,5,6])]
dfsomado = dffiltrado.groupby('contra', as_index=False)[['debito', 'credito', 'saldo']].sum()
dfsomado['nome'] = "SOMA TOTAL"
dffiltrado = pd.concat([dffiltrado, dfsomado], ignore_index=True)
dffiltrado.to_excel(f"{nomearq}-somado.xlsx", index=False)

In [14]:
workbook = load_workbook(f'{nomearq}-resumo.xlsx')
sheet = workbook['RESUMO']
for i in ['A' ,'B' , 'C', 'D']:
    sheet.column_dimensions[i].width = 30
workbook.save(f'{nomearq}-resumo.xlsx')